### Задача

Нарезать видео на кадры из которых будет формироваться датасет для обучения.

### Mount

Подмонтируем папку из сервера к докер контейнеру Jupyter Lab.

In [ ]:
sudo mount --bind /mnt/18tb/360Datasets /var/lib/docker/volumes/jupyterhub-user-{git-username}/_data/constructly_3d_sc_ds/cv/detection/equirectangular/v0.1/data/row/360video

### Import

In [1]:
import cv2
from pathlib import Path
from tqdm.notebook import tqdm

### Paths

Нам необходимы видео `mp4` в формате панорам 360. В папках в формате `mp4` есть видео панорамы 360 и с двумя кругами. Поэтому необходимо вручную отобрать пути до видео с панорамами 360. 

In [2]:
path_videos = [
    '../data/row/360video/360FSK_020524/VID_20240502_101342_00_701.mp4',
    '../data/row/360video/360FSK_020524/VID_20240502_104300_00_702.mp4',
    '../data/row/360video/360FSK-[11-06-24]/Video/VID_20240610_090231_00_141.mp4',
    '../data/row/360video/360FSK-[11-06-24]/Video/VID_20240610_093147_00_142.mp4',
    '../data/row/360video/360FSK-[11-06-24]/Video/VID_20240610_094210_00_143.mp4',
    '../data/row/360video/360FSK-[11-06-24]/Video/VID_20240610_101126_00_144.mp4',
    '../data/row/360video/360Personal/VID_20230617_211942_00_043.mp4',
    '../data/row/360video/360Personal/VID_20230617_222459_00_044.mp4',
    '../data/row/360video/360Personal/VID_20240207_105125_00_044.mp4',
    '../data/row/360video/360Personal/VID_20240214_130309_00_239.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_095811_00_306.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_100804_00_307.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_102848_00_239.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_103011_00_240.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_103124_00_241.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_103410_00_242.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_132037_00_404.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_134954_00_405.mp4',
    '../data/row/360video/360Vnukovo-[01-06-24]/Video/VID_20240602_140611_00_406.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_153455_00_638.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_155217_00_661.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_155735_00_662.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_160554_00_663.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_171425_00_757.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_171656_00_758.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_173506_00_810.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video/VID_20240608_174236_00_639.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video2/VID_20240608_124205_00_076.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video2/VID_20240608_124529_00_077.mp4',
    '../data/row/360video/360Vnukovo-[08-06-24]/Video2/VID_20240608_165139_00_135.mp4',
    '../data/row/360video/360Vnukovo-[15-06-24]/Export/VID_20240615_114927_00_262.mp4',
    '../data/row/360video/360Vnukovo-[15-06-24]/Export/VID_20240615_122020_00_263.mp4',
    '../data/row/360video/360Vnukovo-[15-06-24]/Export/VID_20240615_124935_00_264.mp4',
    '../data/row/360video/Export/VID_20240615_114927_00_262.mp4'
]

### Статистика

Общее время и количество кадров, чтобы понимать сколько получится данных.

In [20]:
total_time, total_frames = 0, 0

for path_video in path_videos:

    cap = cv2.VideoCapture(path_video)
    
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    # Получить частоту кадров (FPS)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Вычислить длительность видео в минутах
    duration_in_sec = frames / fps
    duration_in_min = duration_in_sec / 60
    
    total_time += int(duration_in_min)
    total_frames += frames
    
print(f'Всего минут: {total_time}')
print(f'Всего кадров: {total_frames}')

Всего минут: 320
Всего кадров: 602317.0


### Config

In [3]:
PATH_SAVE = '../data/row/stills'
multiplier = 100

'VID_20240502_101342_00_701'

### Cutting

In [11]:
for path_video in path_videos:
    count = 0
    cap = cv2.VideoCapture(path_video)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    bar = tqdm(total=length // 100, desc=f'Video {Path(path_video).stem}')
    
    while(cap.isOpened() and count < length):
        count += 1
        ret, frame = cap.read()
        if ret:
            if (count % multiplier == 0):
                bar.update(1)
                path_save_img = f'{PATH_SAVE}/{Path(path_video).stem}_{count}.jpg'
                status = cv2.imwrite(path_save_img, frame)

    cap.release()
    bar.close()

Video VID_20240502_101342_00_701:   0%|          | 0/521 [00:00<?, ?it/s]

Video VID_20240502_104300_00_702:   0%|          | 0/276 [00:00<?, ?it/s]

Video VID_20240610_090231_00_141:   0%|          | 0/521 [00:00<?, ?it/s]

Video VID_20240610_093147_00_142:   0%|          | 0/26 [00:00<?, ?it/s]

Video VID_20240610_094210_00_143:   0%|          | 0/521 [00:00<?, ?it/s]

...

Video VID_20240608_165139_00_135:   0%|          | 0/12 [00:00<?, ?it/s]

Video VID_20240615_114927_00_262:   0%|          | 0/398 [00:00<?, ?it/s]

Video VID_20240615_122020_00_263:   0%|          | 0/521 [00:00<?, ?it/s]

Video VID_20240615_124935_00_264:   0%|          | 0/223 [00:00<?, ?it/s]

Video VID_20240615_114927_00_262:   0%|          | 0/398 [00:00<?, ?it/s]

### Resize

Изображения получились очень большими 5888х2944, данные изображения в будущем нужно будет хранить, передавать по сети. Поэтому было принято решение уменьшить изображение до 2048х

In [18]:
import numpy as np

dir_imgs = '../data/row/stills'

In [16]:
path_imgs = [str(p) for p in Path(path_imgs).rglob('*.jpg')]

In [19]:
def get_new_height(img: np.array, new_width: int = 2048) -> int:
    aspect_ratio = img.shape[1] / img.shape[0]
    return int(new_width / aspect_ratio)

In [33]:
new_width = 2048

for path_img in tqdm(path_imgs):
    img = cv2.imread(path_img)
    new_height = get_new_height(img, new_width)
    resized_img = cv2.resize(img, (new_width, new_height))
    
    path_save = f'{dir_save}/{Path(path_img).stem}.jpg'
    cv2.imwrite(path_save, resized_img)

  0%|          | 0/5608 [00:00<?, ?it/s]